In [1]:
##pip install scenariogeneration

In [1]:
#from scenariogeneration import xodr
import os
from scenariogeneration import xosc, prettyprint

/tmp/ipykernel_3226085/1275792747.py:3: DeprecationWarning: STD_ROADMARK_<RoadMarkType> is deprecated and will be removed in a few releases! Use std_roadmark_<RoadMarkType> functions instead.
  from scenariogeneration import xosc, prettyprint


### ParameterDeclarations (document:xosc.utiles)

In [2]:
egoInit = xosc.Parameter(name="EgoVehicle",parameter_type="string",value="pacifica")
egoSpeed = xosc.Parameter(name="EgoSpeed",parameter_type="double",value="60")
egoS = xosc.Parameter(name="Ego_S",parameter_type="double",value="27")

agentInit = xosc.Parameter(name="TargetVehicle",parameter_type="string",value="car_red")
agentSpeed = xosc.Parameter(name="TargetSpeed",parameter_type="double",value="28")
agentS = xosc.Parameter(name="Target_S",parameter_type="double",value="0")

paraList = [egoInit, egoSpeed, egoS, agentInit, agentSpeed, agentS]
paramdec = xosc.ParameterDeclarations()
for i in paraList:
    paramdec.add_parameter(i)

### (Don't modify) CatalogLocations & RoadNetwork (document:xosc.utiles)

In [3]:
# CatalogLocations
catalog = xosc.Catalog()
catalog.add_catalog("VehicleCatalog", "../Catalogs/Vehicles")
catalog.add_catalog("ControllerCatalog", "../Catalogs/Controllers")
#catalog.add_catalog("ControllerCatalog", "../../../workspace/src/esmini/resource/xosc/Catalogs/Controllers")


# RoadNetwork
road = xosc.RoadNetwork(roadfile="../../xodr/itri/hct_6.xodr")

In [18]:
catalog.catalogs

{'VehicleCatalog': '../Catalogs/Vehicles',
 'ControllerCatalog': '../Catalogs/Controllers'}

### Entities (document:xosc.Entities)

In [6]:
#construct CatalogReference
egoObject = xosc.CatalogReference(catalogname="VehicleCatalog", entryname="$EgoVehicle") #xosc.utils
agentObject = xosc.CatalogReference(catalogname="VehicleCatalog", entryname="$TargetVehicle") #xosc.utils

#construct ego controller - ROS
egoControllerProperties = xosc.Properties()
egoControllerProperties.add_property(name="timeGap", value="1.0")
egoControllerProperties.add_property(name="mode", value="override")
egoControllerProperties.add_property(name="setSpeed", value="${$EgoSpeed / 3.6}")
egoController = xosc.Controller(name="ROSController", properties=egoControllerProperties)

# ACC controller
egoControllerProperties = xosc.Properties()
egoControllerProperties.add_property(name="timeGap", value="1.0")
egoControllerProperties.add_property(name="mode", value="override")
egoControllerProperties.add_property(name="setSpeed", value="${$EgoSpeed / 3.6}")
egoController = xosc.Controller(name="ACCController", properties=egoControllerProperties)

# create entity
egoName = "Ego"
agentName = "Agent"
entities = xosc.Entities()
entities.add_scenario_object(name = egoName, entityobject = egoObject, controller = egoController)
entities.add_scenario_object(name = agentName, entityobject = agentObject)

### Storyboard - Init

In [7]:
# initialzie Ego and Agent
init = xosc.Init() #xosc.storyboard
init.add_init_action(egoName, xosc.TeleportAction(xosc.LanePosition(s = "$Ego_S", offset = 0, lane_id = 1, road_id = 72)))
init.add_init_action(
    egoName,
    xosc.AbsoluteSpeedAction("${$EgoSpeed / 3.6}", xosc.TransitionDynamics(xosc.DynamicsShapes.step, xosc.DynamicsDimension.time, 0)))
init.add_init_action(egoName, xosc.ActivateControllerAction(lateral = "true", longitudinal = "true"))
init.add_init_action(egoName, xosc.AcquirePositionAction(xosc.LanePosition(s = 8, offset = 0, lane_id = -1, road_id = 51)))

init.add_init_action(agentName, xosc.TeleportAction(xosc.LanePosition(s = "$Target_S", offset = 0, lane_id = 1, road_id = 89)))



### Storyboard - Event

In [8]:
# 创建一个Trigger-Action，Trigger为仿真一开始即开始，Action为路线规划Route。
egoEvent = xosc.Event("TargetStartWhenEgoCloseToTheJunction", xosc.Priority.overwrite)
egoEvent.add_action("ActivateControllerActionStart", xosc.ActivateControllerAction(lateral = "true", longitudinal = "true"))
egoEvent.add_action("AbsoluteSpeedActionStart",
                     xosc.AbsoluteSpeedAction("${$TargetSpeed / 3.6}",
                     xosc.TransitionDynamics(xosc.DynamicsShapes.step, xosc.DynamicsDimension.time, 0)))

# target position of Agent
egoEvent.add_action("AcquirePositionActionStart", xosc.AcquirePositionAction(xosc.LanePosition(s = 5, offset = 0, lane_id = -1, road_id = 0)))

# Event start trigger
entityCondition = xosc.ReachPositionCondition(xosc.LanePosition(s = 42, offset = 0, lane_id = 1, road_id = 208),tolerance = 2)
egoEvent.add_trigger(xosc.EntityTrigger(name = "EgoEnteringTheJunction", delay = 0,
                                        conditionedge = xosc.ConditionEdge.rising,
                                        entitycondition = entityCondition, triggerentity = egoName, triggeringrule = "any" ) )


### Storyboard - Story & Maneuver

In [9]:
egoManeuver = xosc.Maneuver("MyManeuver")
egoManeuver.add_event(egoEvent)

sb = xosc.StoryBoard(
    init,
    xosc.EntityTrigger(
"ReachDestinationCondition",
        0,
        xosc.ConditionEdge.rising,
        entitycondition = xosc.ReachPositionCondition(xosc.LanePosition(s = 20, offset = 0, lane_id = -1, road_id = 0),tolerance = 1),
        triggerentity = agentName,triggeringpoint="stop"))

# stopTrigger = xosc.EntityTrigger(name = "tetetewtewtwe", delay = 0,
#                                         conditionedge = xosc.ConditionEdge.rising,
#                                         entitycondition = xosc.RelativeDistanceCondition(value = 5, entity = agentName, rule = "lessThan", dist_type = "cartesianDistance", freespace = "false"),
#                                         triggerentity = egoName, triggeringrule = "any",triggeringpoint = "stop")
sb.add_maneuver(egoManeuver, agentName)

### Create Scenario

In [10]:
# 实例化一个Scenario对象 xosc.scenario
sce = xosc.Scenario( 
    name="ITRI DEMO",
    author="YuCheng",
    parameters = paramdec,
    entities=entities,
    storyboard=sb,
    roadnetwork=road,
    catalog=catalog,
    osc_minor_version=0
)
prettyprint(sce.get_element())

#sce.write_xml("/home/yucheng/source_code/lssvip/resources/xosc/itri/testtest.xosc")

<OpenSCENARIO xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="OpenScenario.xsd">
    <FileHeader description="ITRI DEMO" author="YuCheng" revMajor="1" revMinor="0" date="2024-07-05T13:16:53.715673"/>
    <ParameterDeclarations>
        <ParameterDeclaration name="EgoVehicle" parameterType="string" value="pacifica"/>
        <ParameterDeclaration name="EgoSpeed" parameterType="double" value="60"/>
        <ParameterDeclaration name="Ego_S" parameterType="double" value="27"/>
        <ParameterDeclaration name="TargetVehicle" parameterType="string" value="car_red"/>
        <ParameterDeclaration name="TargetSpeed" parameterType="double" value="28"/>
        <ParameterDeclaration name="Target_S" parameterType="double" value="0"/>
    </ParameterDeclarations>
    <CatalogLocations>
        <VehicleCatalog>
            <Directory path="../Catalogs/Vehicles"/>
        </VehicleCatalog>
        <ControllerCatalog>
            <Directory path="../Catalogs/C